# Part 1: Fully Connected Layers

Daniel-Alexandru Bejan (474404)
Patrick Schaper (534366)

The deep learning assignment consists of two parts. At first you will experiment with
fully-connected networks. 

You are going to build multiple neural networks with varying
structures and activation functions for a given dataset.

In the second assignment you are going to build convolutional neural networks for image
classification purposes.

If you want to boost your grade (> 8.0), please have a look at the extra section.